In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torchvision.transforms as T
from torchvision import datasets, models, transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torch.utils.data import random_split
import copy
import numpy as np
from collections import defaultdict
import os

# Import necessary PyTorch libraries
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
from torchvision import datasets, models, transforms

data_transforms = transforms.Compose([

transforms.RandomResizedCrop(224),

transforms.RandomHorizontalFlip(),

transforms.ToTensor(),

transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])


image_datasets = datasets.ImageFolder('/content/drive/My Drive/예선/',data_transforms) # 폴더에 따라 변경

print(image_datasets)
class_names = image_datasets.classes # 클래스 출력

In [ ]:
print(class_names)

In [ ]:
torch.manual_seed(0)

In [ ]:
train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=32,shuffle = True)

inputs, classes = next(iter(train_loader))
print(classes)

In [ ]:
model = models.resnet101(pretrained=True)      # pretain resnet- 101 사용       

number_of_features =  model.fc.in_features
model.fc = nn.Linear(number_of_features, 20)

In [ ]:
from torch.optim import lr_scheduler
import time
USE_GPU = True

if USE_GPU:
    model = model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()

# specify optimizer same as used earlier
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# secify scheduler same as used earlier
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, 'min', patience = 5)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
print(len(train_loader.dataset))

In [ ]:
dataloaders = {}
dataset_sizes = {}
dataloaders['train'] = train_loader
#dataloaders['val'] = test_loader
dataset_sizes['train'] = len(train_loader.dataset)
#dataset_sizes['val'] = len(test_loader.dataset)

In [ ]:
from torch.autograd import Variable
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:               
                pass
                #model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in tqdm(dataloaders[phase]):
                x, y = data;
                inputs = x.squeeze(0).to(device)
                labels = y.to(device)
#                 if phase == 'train':
#                   inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, 0.4, device)
#                   inputs, targets_a, targets_b = Variable(inputs), Variable(labels_a), Variable(labels_b)
                  
                inputs = inputs.to(device)
#                 if phase == 'train':
#                   labels_a = labels_a.to(device)
#                   labels_b = labels_b.to(device)
#                 else:
#                   labels = labels.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
#                     loss_func = mixup_criterion(labels_a,labels_b,lam)
                    
                    _, preds = torch.max(outputs, 1)
                    
                    loss = criterion(outputs, labels)
#                     if phase=='train':
#                       loss = loss_func(criterion,outputs)
#                     else:
#                         loss = criterion(outputs,labels) 

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                #labels11.extend((labels.data).float())
                #guesses.extend(preds.float())
                #print((labels.data).float())
                #print(preds.float())
                #predicted_classes = torch.argmax(y_pred, dim=1) == 0
                #target_classes = self.get_vector(y_batch)
                #target_true += torch.sum(target_classes == 0).float()
                #predicted_true += torch.sum(predicted_classes).float()
                #correct_true += torch.sum(
                #predicted_classes == target_classes * predicted_classes == 0).float()
#                 if phase == 'train':
#                   running_corrects += lam * preds.eq(labels_a.data).sum() + (1 - lam) * preds.eq(labels_b.data).sum()
#                 else:
#                     running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'train':
                scheduler.step(epoch_loss)            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            #recall = correct_true / target_true
            #precision = correct_true / predicted_true
            #f1_score = 2 * precission * recall / (precision + recall)
            #f1_score1 = f1_score(labels11,guesses)
            #print(precision)
            #print(f1_score1)
            # deep copy the model
            torch.save(model.state_dict(), '/content/drive/My Drive/대회2/contest_test_final.pth')           

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print("fin")
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)  

In [ ]:
model1 = models.resnet101()            

number_of_features =  model1.fc.in_features
model1.fc = nn.Linear(number_of_features, 20)

USE_GPU = True
if USE_GPU:
    model1 = model1.cuda()
    
model1.load_state_dict(torch.load("/content/drive/My Drive/대회2/contest_10_12_test_tmp1.pth"))

In [ ]:
test_transforms1 = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
image_datasets = datasets.ImageFolder('/content/drive/My Drive/대회2/test123',test_transforms1)

#train_size = int(0.8 * len(image_datasets))
#test_size = len(image_datasets) - train_size
#train_dataset, test_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32,shuffle = True)
test_loader = torch.utils.data.DataLoader(image_datasets, batch_size=32,shuffle = False)

In [ ]:
#train_size = int(0.8 * len(image_datasets))
#test_size = len(image_datasets) - train_size
#train_dataset, test_dataset = torch.utils.data.random_split(image_datasets, [train_size, test_size])

test_loader = torch.utils.data.DataLoader(image_datasets, batch_size=32,shuffle = False)

#nputs, classes = next(iter(train_loader))
#print(classes)

In [ ]:
labelclass = [3,3,10,11,13, 13,13,13,13,13,13,13,4, 4,4,5,7,7,8,8]
labelclass1 = [5,20,20,14,1,6,9,15,16,17,18,20,2,7,11,8,1,20,6,9]
print(len(labelclass1))

In [ ]:
import csv

with open('/content/drive/My Drive/대회2/output22.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for a in range(3997):
        tmpstr = str(a)+'.jpg'
        tsv_writer.writerow([tmpstr, labelclass[outputList[a]], labelclass1[outputList[a]]])


In [ ]:
import pandas as pd
#dataset = pd.read_csv('/content/drive/My Drive/대회2/test123/test/test.tsv', delimiter='\t', header=None)
dataset = pd.read_csv('/content/drive/My Drive/대회2/output13.tsv', delimiter='\t', header=None)

print(dataset)

In [ ]:
import natsort as nt
from PIL import Image
from torch.autograd import Variable 
def test_model():
    data_transforms = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    #model_ft = torch.load('./models/CustomCNN_model.pth', map_location=device)
    
    path_test = os.path.join(os.getcwd(), '/content/drive/My Drive/대회2/test123/test')
    image_list = nt.natsorted(os.listdir(path_test))
    output_list = []
    for i, images in enumerate(image_list):
        path_test_image = os.path.join(path_test, images)
        image = Image.open(path_test_image)
        image = data_transforms(image)
        image.unsqueeze_(dim=0)
        image = Variable(image)
        image = image.cuda()
        torch.no_grad()
        output = model1(image)
        output = torch.argmax(output, dim=1)
        
        output_list.append(output)
        
    return output_list
output = test_model()

In [ ]:
submission = pd.read_csv('/content/drive/My Drive/대회2/test.tsv')

submission.digit = torch.cat(output).detach().cpu().numpy()
submission.to_csv('/content/drive/My Drive/대회2/output12.tsv', index=False, header=None, sep='\t')

In [ ]:
print(torch.cat(output).detach().cpu().numpy())
outputList = torch.cat(output).detach().cpu().numpy()

In [ ]:
with torch.no_grad():
  for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
      data, target = data.cuda(), target.cuda()          
      ## For 10-crop Testing
      bs, ncrops, c, h, w = data.size()
      # forward pass: compute predicted outputs by passing inputs to the model
      temp_output = model(data.view(-1, c, h, w))
      output = temp_output.view(bs, ncrops, -1).mean(1)
      # calculate the batch loss
      # update average test loss 
      # convert output probabilities to predicted class
      _, pred = torch.max(output, 1)    
      # compare predictions to true label
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
      # calculate test accuracy for each object class


In [ ]:
test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])

In [ ]:
import natsort as nt
from PIL import Image
from torch.autograd import Variable 
def test_model():
    data_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.TenCrop(224),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                      transforms.Lambda(lambda crops: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(crop) for crop in crops]))])
    #model_ft = torch.load('./models/CustomCNN_model.pth', map_location=device)
    
    path_test = os.path.join(os.getcwd(), '/content/drive/My Drive/대회2/test123/test')
    image_list = nt.natsorted(os.listdir(path_test))
    output_list = []
    model1.eval()
    for i, images in enumerate(image_list):
        path_test_image = os.path.join(path_test, images)
        image = Image.open(path_test_image)
        image = data_transforms(image)
        image.unsqueeze_(dim=0)
        image = Variable(image)
        image = image.cuda()
        bs, ncrops, c, h, w = image.size()

        torch.no_grad()
        temp_output = model1(image.view(-1, c, h, w))
        output = temp_output.view(bs, ncrops, -1).mean(1)
        _, pred = torch.max(output, 1)  
        output_list.append(pred)
        
    return output_list
output = test_model()